## **Notebook to Clean and Transform applicable variables**

**Transformations to do:**

- **Date:** convert format to real date '27th November' to '11/27/2025'. Use the Column [Year] to concatenate the year into the normalized Date

- **Year:** Convert to Integer, from '2025.0' to '2025'

- **Type:** Normalize different iterations of the same value, example: 'Unprovoked' & 'unprovoked'

- **Country:** Only 50 nulls, explore those records to see if there is opportunity to extrapolate it from other columns like 'Location' or 'State'

- **State:** N/A

- **Location:** N/A

- **Activity:** Normalize different iterations of the same value. Use NLP to clustter similar activities.

- **Name:** N/A

- **Sex:** Normalize different iterations of the same value

- **Age:** Remove non numerical values, explore those records

- **Injury:** Normalize different iterations of the same value and Explore creating categories (clusters?)

- **Fatal Y/N:** Normalize different iterations of the same value, this must be a binary column, maybe create a different column for values that are not Y/N or delete those records

- **Time:** Normalize to timestamp, create a new column called 'time_cluster' with 'Afternoon/Morning/Night'

- **Species:** Normalize different iterations of the same value, Create a new column with the size when available called 'shark_size' that contains meters, ie. '3m'.

- **Source:** N/A

- **These columns will be excluded from the dataset:** (pdf,href formula, href, Case Number, CaseNumber.1, original order)

### **Libraries**

In [4]:
# Libraries
import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Directories & Files

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
RAW_PATH = os.path.join(BASE_DIR, 'data','raw','GSAF5.csv')
OUT_DIR = os.path.join(BASE_DIR, 'data','processed')
os.makedirs(OUT_DIR, exist_ok=True)

df = pd.read_csv(RAW_PATH)
df.shape()


NameError: name '__file__' is not defined